In [1]:
import pandas as pd
import numpy as np
import gc, sys
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

DATA_FOLDER='../../data/'

In [2]:
#read the recipes file
recipes = pd.read_csv(f'{DATA_FOLDER}/generated/recipes.csv')
recipes.sort_values(['i'], inplace=True)
recipes.head(3)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
46053,40893,0,"[40480, 1454, 16201, 2056, 955, 541, 11332, 82...","[[1424, 8876, 11007], [3484, 21453], [38966, 2...","[40480, 40482, 23667, 11007, 240, 21453, 240, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",0,"[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500...",white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
108526,44394,1,"[40480, 34712, 22683, 11274, 5409, 29868, 40481]","[[5343, 535, 2044, 5409, 7087], [17869, 6020],...","[40480, 40482, 12172, 1281, 5409, 7087, 240, 6...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[912, 7557, 2499, 5382]",devilicious cookie cake delights,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
41248,85009,2,"[40480, 12187, 13995, 571, 14719, 40481]","[[1061, 494, 813, 2141], [31843], [30645, 4785...","[40480, 40482, 2572, 19472, 31757, 512, 823, 4...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",2,"[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169...",baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


In [3]:
# we need to create a collection of ingredients_ids to build vectors of recipes
set_ingredients = set()

for i in tqdm(range(len(recipes['ingredient_ids']))):
    current = [int(x.strip()) for x in recipes['ingredient_ids'].values[i][1:-1].split(',')]
    set_ingredients.update(current)
    
print('Number of ingredients in total :', len(set_ingredients))


Number of ingredients in total : 7993


In [4]:
list_ingredients = list(set_ingredients)

#initialize the matrix
np_matrix = np.zeros((len(recipes), list_ingredients[-1] + 1),dtype=np.uint8)

#populate it
for i in tqdm(range(len(recipes['ingredient_ids']))):
    for x in recipes['ingredient_ids'].values[i][1:-1].split(','):
        np_matrix[i,int(x.strip())] = 1

In [5]:
#wrap the matrix in a pandas dataframe
pd_matrix = pd.DataFrame(np_matrix,columns=range(list_ingredients[-1] + 1), index=recipes['i'].values,dtype=np.uint8)

#add calories column
calories = recipes['calorie_level'].values
pd_matrix['Norm_calorie']= calories

#add time column
time = recipes['minutes'].values
pd_matrix['Norm_time']= time

#add number of steps column
steps = recipes['n_steps'].values
pd_matrix['Norm_steps']= steps

#add number of ingredients column
num_ingr = recipes['n_ingredients'].values
pd_matrix['Norm_num_ingr']= num_ingr

display(pd_matrix)
pd_matrix.shape

,0,1,2,3,4,5,6,7,8,9,...,8017,8018,8019,8020,8021,8022,Norm_calorie,Norm_time,Norm_steps,Norm_num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,495,4,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,5,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,10,3,13
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,40,10,12
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,150,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,45,11,16
178261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13,13,4
178262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,70,9,5
178263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,375,23,10


(178265, 8027)

In [6]:
#check matrix consistensy with user ID 23
assert(pd_matrix.loc[23,389]==1)
assert(pd_matrix.loc[23,7655]==1)
assert(pd_matrix.loc[23,6270]==1)
assert(pd_matrix.loc[23,1527]==1)
assert(pd_matrix.loc[23,3406]==1)
assert(pd_matrix.loc[23,'Norm_time']==61)
assert(pd_matrix.loc[23,'Norm_calorie']==0)
assert(pd_matrix.loc[23,'Norm_steps']==6)
assert(pd_matrix.loc[23,'Norm_num_ingr']==5)

In [7]:
'''
# Identify missing values 
import math
w=float('nan')
math.isnan(w)
pd_matrix.isna().sum()
pd_matrix.isnull().sum().sum()'''

"\n# Identify missing values \nimport math\nw=float('nan')\nmath.isnan(w)\npd_matrix.isna().sum()\npd_matrix.isnull().sum().sum()"

In [8]:
def NormalizeData(matrix, PCA_comp = None, ScaleCat = False):
    '''
    Normalize data using a standard scaler
    train:
        dataframe that will be use to fit and transformed by the scaler and PCA
    CVorTest:
        dataframe that will be transformed the scaler and PCA
    PCA_comp:
        Number of PCA components to keep, if None, PCA not applied
    ScaleCat:
        Scale or not the categorical columns with the standard scaler
    '''
    
    sc = StandardScaler()
    
    if ScaleCat:
        scale_columns = matrix.columns
    else:
        scale_columns = [col for col in matrix.columns[matrix.columns.str.startswith('Norm_', na=False)]]
          
    #perform feature scaling    
    print('Nb columns to scale :', len(scale_columns))
    matrix.loc[:, scale_columns] = sc.fit_transform(matrix.loc[:, scale_columns]) 
    
    if PCA_comp is None:
        return matrix
    
    pca = PCA(PCA_comp)
    matrix = pca.fit_transform(matrix)
    
    return matrix

In [9]:
#Normalize Data
pd_matrix = NormalizeData(pd_matrix, None, False)

Nb columns to scale : 4


In [10]:
pd_matrix

,0,1,2,3,4,5,6,7,8,9,...,8017,8018,8019,8020,8021,8022,Norm_calorie,Norm_time,Norm_steps,Norm_num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.002290,-1.228415,-0.005578
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.002384,-0.982453,-1.556352
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.002386,-1.474377,1.235041
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.172849,-0.002380,0.247357,0.924886
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.002358,1.231205,2.165505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.002379,0.493319,2.165505
178261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.002385,0.985243,-1.556352
178262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.002374,0.001395,-1.246197
178263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.002314,3.444863,0.304577


In [17]:
from scipy.sparse.linalg import svds

#takes around 20 minutes
U, S, V = svds(pd_matrix.values.astype(np.float32),k=2000)

KeyboardInterrupt: 

In [ ]:
reduced_matrix = U * S
print('shape of reduced matrix :', reduced_matrix.shape)

In [11]:
def OneVSAllCosineSimilarity(oneId, featuresMatrix):
    """For a matrix of features, computes the cosine similarity for one index *oneId* against all ids 
       including *oneId*
        oneId :  
            int, the id on wich the cosine similarity must be compute
        featuresMatrix: 
            numpy array, matrix of features : items x features
    :Returns:
         A array of cosine similarity of size (1, items)
    :Example:
        cosineSimilarity = OneVSAllCosineSimilarity(0, pd_matrix)
    """
    oneVector = featuresMatrix[oneId,:].reshape(1, -1)
    
    return cosine_similarity(oneVector, featuresMatrix)[0]

In [12]:
#execute cosine similarity on user #7
result = OneVSAllCosineSimilarity(99787, pd_matrix.values)
#result_reduced = OneVSAllCosineSimilarity(7, reduced_matrix)
result

array([ 0.05100897,  0.31595139, -0.05430221, ...,  0.25419721,
       -0.06942113,  0.06111385])

In [13]:
def GetClosestRecipes(oneId, cosine_vector, N, verbose=False):
    """Return the topN recipes in cosine_vector considering oneId in cosine_vector
        oneId :  
            int, the id on wich the cosine similarity has been compute
        cosine_vector: 
            numpy array, array of cosine vector for all recipes
        N:
            integer, number of top N recipes to return
        verbose:
            Boolean, if true, print topN recipes info
    :Returns:
         two arrays. An array of topN recipes ID (i) and a array of topN scores
    :Example:
        GetClosestRecipes(0, result, 20)
    """
    records = recipes[recipes['i']==oneId]
    rec = records[['name', 'ingredients', 'minutes', 'calorie_level', 'n_steps', 'n_ingredients']]
    origin_reco = [oneId]    
    locationOneId = np.where(np.argsort(cosine_vector)[::-1] == oneId)[0][0]
    topNrecipes = None
    
    if locationOneId >= N:
        topNrecipes = np.argsort(cosine_vector)[::-1][:N]
        topNScore = np.sort(cosine_vector)[::-1][:N]
    else:
        topNrecipes = np.argsort(cosine_vector)[:(-N-2):-1][np.arange(N+1)!=locationOneId]
        topNScore = np.sort(cosine_vector)[:(-N-2):-1][np.arange(N+1)!=locationOneId]
        
    
    toprecords = recipes[recipes['i'].isin(topNrecipes)]
    
    if verbose:
        print(f'The recipe you are looking for ({oneId}) is:')
        pd.set_option('display.max_colwidth', -1)
        display(rec)
        df_closest = pd.DataFrame({'score':topNScore}, index = topNrecipes)
        print(f'the top {N} closest recipes are:')
        topN = toprecords[['i', 'name', 'ingredients', 'minutes', 'calorie_level', 'n_steps', 'n_ingredients']]
        closest_reco = df_closest.join(topN.set_index('i'))
        closest_reco.sort_values('score', inplace=True, ascending=False)
        display(closest_reco)
    gc.collect()
    
    return topNrecipes, topNScore

GetClosestRecipes(99787, result, 5, True)
#GetClosestRecipes(7, result_reduced, 5, True)

The recipe you are looking for (99787) is:


,name,ingredients,minutes,calorie_level,n_steps,n_ingredients
114613,to die for crock pot roast,"['beef roast', 'brown gravy mix', 'dried italian salad dressing mix', 'ranch dressing mix', 'water']",545,1,7,5


the top 5 closest recipes are:


,score,name,ingredients,minutes,calorie_level,n_steps,n_ingredients
53731,0.962496,easy company pot roast,"['beef roast', 'brown gravy mix', 'dried italian salad dressing mix', 'dry ranch dressing mix', 'water']",245,1,4,5
135043,0.860805,three packs and a roast,"['beef roast', 'ranch dressing mix', 'italian salad dressing mix', 'brown gravy mix', 'water']",485,0,4,5
175604,0.821009,another crock pot roast,"['beef roast', 'ranch dressing mix', 'liquid smoke', 'brown gravy mix', 'water']",1445,0,5,5
109423,0.765186,crock pot italian beef sandwiches,"['beef roast', 'good seasons italian dressing', 'garlic', 'pepperoncini peppers', 'water']",490,1,7,5
93246,0.763927,italian beef au jus,"['boneless beef roast', 'au jus mix', 'italian salad dressing', 'beef consomme', 'water']",610,1,5,5


(array([ 53731, 135043, 175604, 109423,  93246]),
 array([0.96249561, 0.86080496, 0.82100894, 0.76518563, 0.76392722]))

In [33]:
def printFinalReco(dict_reco):
    """Print the topN recipes recommended
        dict_reco:  dictionnary containning the recipe id, the similarity score and the reference recipe id
    :Returns:
         one table containing the recomended recipes with their reference recipe and the similarity score
    :Example:
        printFinalReco(reco)
    """
   
    recipe_id = []
    ref_id = []
    scores = []
    
    for i in range(len(dict_reco)):
        recipe_id.append(dict_reco[i][0])
        scores.append(dict_reco[i][1][0])
        ref_id.append(dict_reco[i][1][1])
    
    refs_name = []
    recipe_name = []
    for i in range(len(dict_reco)):
        a = recipes['name'][recipes['i']==recipe_id[i]].values
        recipe_name.append(a[0])
        b = recipes['name'][recipes['i']==ref_id[i]].values
        refs_name.append(b[0])
    
    recipe_id_name = pd.DataFrame({'score':scores, 'recipe name':recipe_name}, index = recipe_id)
    refs_id_name = pd.DataFrame({'ref name':refs_name, 'id refs':ref_id}, index = recipe_id)
    final_reco = recipe_id_name.join(refs_id_name)
    
    return final_reco           

In [34]:
#read the train file to get user's data
interactions = pd.read_csv('../../data/generated/interactions_full.csv')
interaction = pd.read_csv('../../data/generated/interactions_test.csv')
def getCBFRecommandation(userId, topN, listRecipesId, ing_matrix):
    """Return the topN recipes in cosine_vector considering oneId in cosine_vector
        oneId :  
            int, the id on wich the cosine similarity has been compute
        cosine_vector: 
            numpy array, array of cosine vector for all recipes
        N:
            integer, number of top N recipes to return
        verbose:
            Boolean, if true, print topN recipes info
    :Returns:
         two arrays. A array of topN recipes ID (i) and a array of topN scores
    :Example:
        GetClosestRecipes(0, result, 20)
    """
    dict_recipes = dict()

    for i in tqdm(listRecipesId):
        similarity = OneVSAllCosineSimilarity(i, ing_matrix)
        closest_recipes, scores = GetClosestRecipes(i, similarity, topN, True)
        for closest_recipe, score in zip(closest_recipes, scores):
            if closest_recipe not in interaction[interaction['u']==userId]['i'].values: #exclude recipes already performed by user
                if (closest_recipe not in dict_recipes) or (dict_recipes[closest_recipe] < score):
                    dict_recipes[closest_recipe] = (score,i)

    dict_recipes = sorted(dict_recipes.items(), key=lambda k: k[1][0], reverse=True)
    TNR = printFinalReco(dict_recipes[:topN])
    return TNR

In [56]:
# Add weights on features
# OPTIONNAL !!
def AddFeaturesWeight(datframe):
    datframe.iloc[:, : 8022]= (datframe.iloc[:, : 8022])*4
    datframe['Norm_time'] = datframe['Norm_time']*3
    datframe['Norm_steps'] = datframe['Norm_steps']*2
    return datframe
pd_matrix1 = AddFeaturesWeight(pd_matrix)
pd_matrix1

,0,1,2,3,4,5,6,7,8,9,...,8017,8018,8019,8020,8021,8022,Norm_calorie,Norm_time,Norm_steps,Norm_num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.020612,-2.456831,-0.005578
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.021452,-1.964906,-1.556352
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.021470,-2.948755,1.235041
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.172849,-0.021417,0.494714,0.924886
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.021222,2.462410,2.165505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.021408,0.986638,2.165505
178261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.090600,-0.021465,1.970486,-1.556352
178262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.021364,0.002790,-1.246197
178263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.436299,-0.020824,6.889727,0.304577


In [35]:
def initiateCBF(userId, TopN, n_reference):
    """ Initiate the CBF and runs it. Return the TopN recipes entered for the userId selected
        and apply the CBF on the number of recipes entered
        userId :  
            integer, id of the user to perform CBF on
        TopN: 
            integer, number of recipes you want to be reccomended to the selected user 
        n_reference:
            integer, number of recipes to perform CBF on (better not to be over 5)
    :Returns:
         one dataframe. A dataframe of TopN recipes, their reference recipe and the similarity score
    :Example:
        initiateCBF(10,5,5)
    """
    
    ratings=np.where(interaction[interaction['u']==userId]['rating']>=4)
    reccomendation = getCBFRecommandation(userId, TopN, interaction[interaction['u']==userId].iloc[ratings]['i'].values[:n_reference],pd_matrix.values)
    print(f'The recomendation given by CBF for user ({userId}) is:')
    return reccomendation
    

In [36]:
initiateCBF(2,10,1)

The recipe you are looking for (118163) is:


,name,ingredients,minutes,calorie_level,n_steps,n_ingredients
23715,tommy fries,"['ranch dressing', 'bacon', 'cheddar cheese', 'potatoes', 'salt and pepper']",60,2,11,5


the top 10 closest recipes are:


,score,name,ingredients,minutes,calorie_level,n_steps,n_ingredients
31571,0.818171,bacon cheddar potato tart michael smith,"['baking potatoes', 'bacon', 'cheddar cheese', 'salt and pepper']",190,2,13,4
176309,0.796160,mom s camp fire potatoes,"['potatoes', 'cheddar cheese', 'green onions', 'margarine', 'bacon', 'salt and pepper']",55,2,8,6
142804,0.774249,cheesy bacon ranch pasta salad,"['cheese tortellini', 'bacon', 'cherry tomatoes', 'cheddar cheese', 'ranch dressing']",20,2,11,5
18295,0.764499,slow cooker cheesy bacon ranch potatoes,"['potatoes', 'onion', 'bacon', 'shredded cheddar cheese', 'ranch dressing mix', 'butter', 'green onion']",260,2,11,7
154045,0.747760,arkansas breakfast casserole,"['bacon', 'green pepper', 'onions', 'potatoes', 'sharp cheddar cheese', 'eggs', 'salt and pepper']",30,2,9,7
46474,0.740876,cheesy italian potatoes,"['russet potatoes', 'italian salad dressing mix', 'butter', 'bacon', 'cheddar cheese']",60,2,8,5
113133,0.732190,contest winner twice baked potato casserole,"['potatoes', 'bacon', 'sour cream', 'onion', 'butter', 'cheddar cheese', 'salt and pepper']",70,2,8,7
116112,0.720037,parsnips potatoes and bacon,"['parsnip', 'potatoes', 'bacon', 'chicken stock', 'salt and pepper', 'cheddar cheese']",65,2,5,6
48295,0.719324,german fried potatoes,"['potatoes', 'bacon', 'onions', 'fresh parsley', 'salt and pepper', 'olive oil']",60,2,12,6
140725,0.717650,bacon cheeseburger chicken,"['olive oil', 'chicken breast halves', 'bacon', 'teriyaki sauce', 'ranch salad dressing', 'cheddar cheese']",375,2,11,6



The recomendation given by CBF for user (2) is:


,score,recipe name,ref name,id refs
31571,0.818171,bacon cheddar potato tart michael smith,tommy fries,118163
176309,0.796160,mom s camp fire potatoes,tommy fries,118163
142804,0.774249,cheesy bacon ranch pasta salad,tommy fries,118163
18295,0.764499,slow cooker cheesy bacon ranch potatoes,tommy fries,118163
154045,0.747760,arkansas breakfast casserole,tommy fries,118163
46474,0.740876,cheesy italian potatoes,tommy fries,118163
113133,0.732190,contest winner twice baked potato casserole,tommy fries,118163
116112,0.720037,parsnips potatoes and bacon,tommy fries,118163
48295,0.719324,german fried potatoes,tommy fries,118163
140725,0.717650,bacon cheeseburger chicken,tommy fries,118163
